In [2]:
import pandas as pd
import numpy as np
from pprint import pprint

In [3]:
from pymongo import MongoClient

In [4]:
pd.set_option('display.max_columns', 0)

In [5]:
def carregarDF():
    
    df_sub = pd.read_csv(f"../../Enem2021/dados/MICRODADOS_ENEM_2021.csv", sep=";", encoding="ISO-8859-1")
    #df = pd.read_csv(f"./data/tb{table_load.lower()}.csv", sep=";", encoding="latin1", error_bad_lines=False)
    
    df_sub.fillna(0, inplace=True)
    
    return df_sub 

def carregarDFamostra():
    
    df_sub = pd.read_csv(f"../dados/base_enem_amostra.csv"
                         , sep=";"
                         , encoding="ISO-8859-1")
    
    return df_sub

def gerarDFamostra(df_sub,qtd_rows):
    
    return df_sub.sample(qtd_rows,random_state = 1)

def gravarDFamostra(df_sub):
    
    df_sub.to_csv(f"../dados/base_enem_amostra.csv"
                  , sep = ";"
                  , encoding="ISO-8859-1"
                  , index=False)

In [ ]:
%%time
# Carrega o dataframe com as informações do ENEM
df = carregarDF()
df.shape

In [ ]:
%%time
# Gera a base de amostras
df_amostra = gerarDFamostra(df, 1000000)

# Grava a base de amostras em arquivo
gravarDFamostra(df_amostra)

In [6]:
%%time
# Carrega a base de amostras, para trabalhar com ela no artigo
df = carregarDFamostra()
df.shape

CPU times: user 7.14 s, sys: 1.08 s, total: 8.22 s
Wall time: 8.24 s


(1000000, 76)

In [7]:
%%time
df['TP_PRESENCA_CN'] = df['TP_PRESENCA_CN'].astype(int)
df['TP_PRESENCA_CH'] = df['TP_PRESENCA_CH'].astype(int)
df['TP_PRESENCA_LC'] = df['TP_PRESENCA_LC'].astype(int)
df['TP_PRESENCA_MT'] = df['TP_PRESENCA_MT'].astype(int)
df['TP_STATUS_REDACAO'] = df['TP_STATUS_REDACAO'].astype(int)

# Converte a coluna QTD MORADORES para FLOAT
df['Q005'] = df['Q005'].astype(float) # QTD MORADORES

df['NU_NOTA_REDACAO'] = df['NU_NOTA_REDACAO'].astype(float)

# Converte os códigos das provas para INT
df['CO_PROVA_LC'] = df['CO_PROVA_LC'].astype(int)
df['CO_PROVA_CH'] = df['CO_PROVA_CH'].astype(int)
df['CO_PROVA_CN'] = df['CO_PROVA_CN'].astype(int)
df['CO_PROVA_MT'] = df['CO_PROVA_MT'].astype(int)

CPU times: user 295 ms, sys: 115 ms, total: 410 ms
Wall time: 409 ms


In [ ]:
# df_enem_2021[2000000:].to_csv('../Enem2021/dados/df_enem_2021_2M-end.csv'
#                                , index=False)

In [ ]:
df.info(verbose=True)


In [ ]:
# #df_enem_2021_inicio = df_enem_2021[0:200000]
# df_enem_2021_resto = df_enem_2021[200000:2000000]
# df_enem_2021_resto_mesmo = df_enem_2021[2000000:]

In [ ]:
# print(df_enem_2021_inicio.shape)
# print(df_enem_2021_resto.shape)
# print(df_enem_2021_resto_mesmo.shape)

In [ ]:
#df_enem_2021_resto.to_csv('df_enem_2021_resto_mesmo.csv', index=False)

In [8]:
def get_connection():
    # Provide the mongodb atlas url to connect python to mongodb using pymongo
    CONNECTION_STRING = "mongodb://mongo1:27017,mongo2:27018,mongo3:27019/?replicaSet=myReplicaSet"

    # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
    client = MongoClient(CONNECTION_STRING)

    return client
    # Create the database for our example (we will use the same database throughout the tutorial


In [9]:
def create_database(client_sub, dbname_sub):
    
    return client_sub[dbname_sub]

In [13]:
client = get_connection()
print(client)

MongoClient(host=['mongo1:27017', 'mongo3:27019', 'mongo2:27018'], document_class=dict, tz_aware=False, connect=True, replicaset='myReplicaSet')


In [14]:
dbname = create_database(client, 'db_enem_2021')
print(dbname)

Database(MongoClient(host=['mongo1:27017', 'mongo3:27019', 'mongo2:27018'], document_class=dict, tz_aware=False, connect=True, replicaset='myReplicaSet'), 'db_enem_2021')


In [15]:
collection_name = dbname["tb_geral"]
print(collection_name)

Collection(Database(MongoClient(host=['mongo1:27017', 'mongo3:27019', 'mongo2:27018'], document_class=dict, tz_aware=False, connect=True, replicaset='myReplicaSet'), 'db_enem_2021'), 'tb_geral')


In [ ]:
%%time
# Converte o dataframe para o formato [{column -> value}, … , {column -> value}]
df = df.to_dict('records')



In [ ]:
%%time
# Remove nulos
#df_enem_2021_dict_notna = [ {k:v for k,v in m.items() if pd.notnull(v)} for m in df_enem_2021.to_dict('records')]
#df_enem_2021_dict_notna

In [ ]:
len(df)

In [ ]:
%%time

client = get_connection()
dbname = create_database(client, 'db_enem_2021')
collection_name = dbname["enem_2021"]

count = 4
x = 200000 * count

inicio = 0 + x
fim = 200000 + x

print(f"Inserção dos registros (count={count}) {inicio:,.0f} a {fim:,.0f}")
print()
collection_name.insert_many(df[inicio:fim])

client.close()


In [ ]:
#dbname.drop_collection(collection_name)

## Delete 

In [ ]:
print(collection_name)

In [ ]:
%%time

client = get_connection()
dbname = create_database(client, 'db_enem_2021')
collection_name = dbname["enem_2021"]

collection_name.delete_many( {"$or": [{ 'TP_PRESENCA_CN' : {"$ne" : 1 } }
                                         ,{ 'TP_PRESENCA_CH' : {"$ne" : 1 } }
                                         ,{ 'TP_PRESENCA_LC' : {"$ne" : 1 } }
                                         ,{ 'TP_PRESENCA_MT' : {"$ne" : 1 } }
                                         ,{ 'TP_STATUS_REDACAO' : {"$ne" : 1 }}]})

client.close()


### UPDATE
#### Atualiza a coluna REGIAO pela região do local de prova do inscrito

In [ ]:
%%time
result = collection_name.update_many(
    filter={} ,
    update=[
        {
            "$set": {
                "regiao": None,
            }
       },
   ],
)

In [ ]:
%%time
client = get_connection()
dbname = create_database(client, 'db_enem_2021')
collection_name = dbname["enem_2021"]

collection_name.updateMany(
    filter={} ,
   { "$unset": { "regiao": ""} })
client.close()

In [ ]:
%%time

client = get_connection()
dbname = create_database(client, 'db_enem_2021')
collection_name = dbname["enem_2021"]

# result = collection.update_many({'NU_INSCRICAO': '210053865474'}, {"$set":{"created": "$Q025"}})
#result.matched_count


result = collection_name.update_many(
    filter={"SG_UF_PROVA": {"$in": ['DF', 'GO', 'MS', 'MT']}} ,
    update=[
        {
            "$set": {
                "regiao": 'CO',
            }
       },
   ],
)

print(f'Update Centro-oeste = {result.matched_count}')

result = collection_name.update_many(
    filter={"SG_UF_PROVA": {"$in": ['MA', 'PB', 'PE', 'PI', 'RN', 'SE', 'AL', 'BA', 'CE']}} ,
    update=[
        {
            "$set": {
                "regiao": 'NE',
            }
       },
   ],
)

print(f'Update Nordeste = {result.matched_count}')

result = collection_name.update_many(
    filter={"SG_UF_PROVA": {"$in": ['AC', 'PA', 'RO', 'RR', 'TO', 'AM', 'AP']}} ,
    update=[
        {
            "$set": {
                "regiao": 'N',
            }
       },
   ],
)

print(f'Update Norte = {result.matched_count}')


result = collection_name.update_many(
    filter={"SG_UF_PROVA": {"$in": ['ES', 'MG', 'RJ', 'SP']}} ,
    update=[
        {
            "$set": {
                "regiao": 'SE',
            }
       },
   ],
)

print(f'Update Sudeste = {result.matched_count}')

result = collection_name.update_many(
    filter={"SG_UF_PROVA": {"$in": ['RS', 'SC', 'PR']}} ,
    update=[
        {
            "$set": {
                "regiao": 'S',
            }
       },
   ],
)

print(f'Update Sul = {result.matched_count}')
client.close()


In [ ]:
# #collection_name.count_documents({})
# collection_name.find_one({})

### SELECT 1
#### Selecionar 10 inscritos, por número de inscrição, fazendo um loop e NU_INSCRICAO aleatório, com random_state do sample igual ao contador do loop

In [ ]:
%%time
# Carrega a base de amostras, para trabalhar com ela no artigo
df = carregarDFamostra()
df.shape

In [ ]:
df = df[(df['TP_PRESENCA_CN'] == 1) & 
   (df['TP_PRESENCA_CH'] == 1) & 
   (df['TP_PRESENCA_LC'] == 1) & 
   (df['TP_PRESENCA_MT'] == 1) & 
   (df['TP_STATUS_REDACAO'] == 1)]

In [ ]:
df['NU_INSCRICAO'].head(2)

In [ ]:
client = get_connection()
dbname = create_database(client, 'db_enem_2021')
collection_name = dbname["enem_2021"]

random_nu_inscricao = 210051738295
cur = collection_name.find({"NU_INSCRICAO" : random_nu_inscricao})

for doc in cur:
    print(doc)

client.close()

In [ ]:
%%time

for i in range(10):
    random_nu_inscricao = df.sample(1, random_state=i)['NU_INSCRICAO'].item()
    
    cur = collection_name.find({"NU_INSCRICAO" : random_nu_inscricao})
    
    #print(len(list(cur)))
    

### SELECT 2
#### "Selecionar todos os documentos em condição de 3 colunas 
#### (inscritos da região ""Sudeste"" e pais com curso superior ou pós graduação)"

In [ ]:
%%time

client = get_connection()
dbname = create_database(client, 'db_enem_2021')
collection_name = dbname["enem_2021"]

cur = collection_name.find(
     { "$and": [{"$or": [{ "Q001": {"$in": ["F", "G"]} },
                        { "Q002": {"$in": ["F", "G"]} }]},
                 {"regiao" : "SE" },
                 {"NU_NOTA_REDACAO" : { "$gt": 640}  }]
     }
    )


client.close()

### SELECT 3
#### Fazer um agrupamento de formação dos pais e computador e contar quantos inscritos em cada grupo, com order by

In [26]:
%%time

client = get_connection()
dbname = create_database(client, 'db_enem_2021')
collection_name = dbname["enem_2021"]


cur = collection_name.aggregate([
    {
        "$group" : {
            "_id" : { "Q001" : "$Q001"
                     , "Q002" : "$Q002"
#                     , "Q002" : "$Q002"
                   , "Q024" : "$Q024"},
            "count" : { "$sum" : 1}
        }},
        {"$sort" : {"_id" : 1}
            
        }])

# for doc in cur: 
#     pprint(doc)

client.close()

CPU times: user 18.5 ms, sys: 8.67 ms, total: 27.2 ms
Wall time: 1.77 s


### SELECT 4
####  Consulta com GROUP BY, HAVING e SORT
#### Recupera as regiões que possuem mais de 200.000 inscritos, com o resultado ordenado pelo nome da região"

In [ ]:
%%time

client = get_connection()
dbname = create_database(client, 'db_enem_2021')
collection_name = dbname["enem_2021"]


cur = collection_name.aggregate([
    {
        "$group" : {
            "_id" : { "regiao" : "$regiao"},
            "count" : { "$sum" : 1}
        }},
        { '$match': { 'count': { '$gt': 200000 } } },
        {"$sort" : {"_id" : 1}
            
        }])

#for doc in cur: 
#    print(doc)

client.close()